In [1]:
__author__ = 'Monika Soraisam'
__email__ = 'monika.soraisam@noirlab.edu'

In [2]:
import os
from pathlib import Path
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime

# for bokeh apps to load from the jupyter notebook on a different browser tab
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Now import the DRAGONS libraries 
import astrodata
import gemini_instruments
from recipe_system import cal_service
from gempy.adlibrary import dataselect
from gempy.utils import logutils

from recipe_system.reduction.coreReduce import Reduce
from gempy.scripts import showpars
from gempy.utils.showrecipes import showrecipes
from gempy.utils.showrecipes import showprims

Ginga not installed, use other viewer, or no viewer


# GMOS longslit example

In [4]:
data_path = "/data/goats_dev_data/example_data/data/ZTF18acppavh/GEM"
obsid = 'GN-2018B-Q-315-11'

### Generate filelist for the data reduction

In [6]:
def generate_filelists(location, obsid):
    """
    Parameters
    ----------
    location: str
        Root folder where the Gemini data for a given target is located
    obsid: str
        Gemini observation ID 
    """

    
    all_files = [str(pp) for pp in list(Path(location+"/"+obsid).glob('*.fits'))]
    all_files.sort()
    print (f'The total number of files for observation ID {obsid} is {len(all_files)}')

    cal_file_tags = ['BIAS','DARK','FLAT','ARC','PINHOLE','RONCHI','FRINGE'] #fetched from Obs Type search field on GOA, which is relevant for DRAGONS

    meta_keys = cal_file_tags + ['BPM','standard','object','unknown']
    all_meta = {}

    for K in meta_keys:
        all_meta[K] = {'file':[],
                        'group_id':[],
                        'exp':[],
                        'objname':[],
                        'wave':[],
                        'waveband':[],
                        'date':[],
                        'roi':[],
                        }

    object_files = []
    for i,F in enumerate(all_files):
        ad = astrodata.open(F)

        K = "unknown"
        if "BPM" in ad.tags:
            K = "BPM"
        elif "PREPARED" in ad.tags:
            continue
        elif ("STANDARD" in ad.tags or ad.observation_class()=="partnerCal" or ad.observation_class()=="progCal") and ("UNPREPARED" in ad.tags) and (ad.observation_type()=="OBJECT"):
            K = "standard"            
        elif "CAL" in ad.tags and "UNPREPARED" in ad.tags:
            for tag in cal_file_tags:
                if tag in ad.tags:
                    K = tag
        elif ad.observation_class()=="science" and "UNPREPARED" in ad.tags:
            K = "object"
            
        all_meta[K]['file'].append(F)
        # group_id seems to be not implemented for GNIRS yet
        if "GNIRS" in ad.instrument():
            all_meta[K]['group_id'].append(None)
        else:
            all_meta[K]['group_id'].append(ad.group_id())
        all_meta[K]['exp'].append(ad.exposure_time())
        all_meta[K]['objname'].append(ad.object())
        all_meta[K]['wave'].append(ad.central_wavelength())
        all_meta[K]['waveband'].append(ad.wavelength_band())
        all_meta[K]['date'].append(ad.ut_date())
        all_meta[K]['roi'].append(ad.detector_roi_setting()) 
        #print (F.split('/')[-1], ad.object(), ad.tags)
    
    return all_meta


In [7]:
all_meta = generate_filelists(data_path, obsid)

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}")


The total number of files for observation ID GN-2018B-Q-315-11 is 121
There are 100 files for observation type BIAS
There are 5 files for observation type FLAT
There are 6 files for observation type ARC
There are 1 files for observation type BPM
There are 2 files for observation type standard
There are 4 files for observation type object
There are 2 files for observation type unknown


In [8]:
DF_bias = pd.DataFrame(all_meta['BIAS'])
DF_flat = pd.DataFrame(all_meta['FLAT'])
DF_bpm = pd.DataFrame(all_meta['BPM'])
DF_arc = pd.DataFrame(all_meta['ARC'])
DF_object = pd.DataFrame(all_meta['object'])
DF_standard = pd.DataFrame(all_meta['standard'])


In [9]:
print (f"There are {len(np.unique(DF_object['objname'].values))} science targets in this observation set, namely {np.unique(DF_object['objname'].values)}")

There are 1 science targets in this observation set, namely ['IRAS 05414+5840']


In [10]:
DF_all_objects = pd.concat([DF_object, DF_standard], ignore_index=True) # for GMOS this dataframe concatenation will only include science objects (i.e., standard and object)

In [11]:
np.unique(DF_all_objects['date'].values)

array([datetime.date(2018, 8, 20), datetime.date(2018, 11, 14),
       datetime.date(2018, 11, 15)], dtype=object)

In [12]:
temp = pd.DataFrame(np.reshape(DF_bias['date'].values, (-1,1)) - np.unique(DF_all_objects['date'].values))
## convert each column to int from timedelta
for K in temp.columns:
    temp[K] = temp[K].dt.days

In [13]:
temp ## each column corresponds to a unique date of interest

,0,1,2
0,-8,-94,-95
1,-8,-94,-95
2,-8,-94,-95
3,-8,-94,-95
4,-8,-94,-95
...,...,...,...
95,92,6,5
96,92,6,5
97,92,6,5
98,92,6,5


In [14]:
## create a dataframe, which is a copy of "temp" but set all values to False
mask = temp.copy(deep=True) 
mask.loc[:,:] = 0
mask = mask.astype(bool)

## Now find the first 20 bias frames closest in time to a given date of interest 
for col in temp.columns:
    idx = np.argsort(np.abs(temp[col]).values)
    mask[col].values[idx[0:20]] = True

In [15]:
mask

,0,1,2
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
95,False,False,False
96,False,False,False
97,False,False,False
98,False,False,False


In [16]:
mask_final = mask.iloc[:].sum(axis=1).astype(bool)

In [17]:
DF_bias = DF_bias[mask_final]
len(DF_bias), len(mask_final), np.sum(mask_final)

(45, 100, 45)

# GHOST example

**Bias filtering for GHOST** -- unlike GMOS spectroscopy where master bias frames are generated for use with the science target and standard star, for GHOST which is an echelle spectrograph, specific master bias frames are required for science target, standard star, flat and arc (additionally also for slit-viewer camera, but it will be automatically taken care of). So the date filtering for bias needs to account for all of these.


In [23]:
data_path = "/data/goats_dev_data/example_data/data/ZTF18acppavh/GEM"
obsid = 'GS-2024A-Q-414-13'

In [24]:
all_meta = generate_filelists(data_path, obsid)

The total number of files for observation ID GS-2024A-Q-414-13 is 161


In [25]:
for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}")

There are 150 files for observation type BIAS
There are 2 files for observation type FLAT
There are 2 files for observation type ARC
There are 2 files for observation type BPM
There are 4 files for observation type standard
There are 1 files for observation type object


In [28]:
DF_bias = pd.DataFrame(all_meta['BIAS'])
DF_flat = pd.DataFrame(all_meta['FLAT'])
DF_bpm = pd.DataFrame(all_meta['BPM'])
DF_arc = pd.DataFrame(all_meta['ARC'])
DF_object = pd.DataFrame(all_meta['object'])
DF_standard = pd.DataFrame(all_meta['standard'])


In [29]:
DF_all_objects = pd.concat([DF_standard, DF_object, DF_flat, DF_arc], ignore_index=True)

In [30]:
np.unique(DF_all_objects['date'].values)

array([datetime.date(2023, 12, 18), datetime.date(2024, 3, 22),
       datetime.date(2024, 5, 10), datetime.date(2024, 5, 12),
       datetime.date(2024, 5, 13)], dtype=object)

In [31]:
temp = pd.DataFrame(np.reshape(DF_bias['date'].values, (-1,1)) - np.unique(DF_all_objects['date'].values))
## convert each column to int from timedelta
for K in temp.columns:
    temp[K] = temp[K].dt.days

In [32]:
temp ## each column corresponds to a unique date of interest

,0,1,2,3,4
0,-3,-98,-147,-149,-150
1,-3,-98,-147,-149,-150
2,-1,-96,-145,-147,-148
3,-1,-96,-145,-147,-148
4,-1,-96,-145,-147,-148
...,...,...,...,...,...
145,148,53,4,2,1
146,148,53,4,2,1
147,148,53,4,2,1
148,148,53,4,2,1


In [33]:
## create a dataframe, which is a copy of "temp" but set all values to False
mask = temp.copy(deep=True) 
mask.loc[:,:] = 0
mask = mask.astype(bool)

## Now find the first 20 bias frames closest in time to a given date of interest 
for col in temp.columns:
    idx = np.argsort(np.abs(temp[col]).values)
    mask[col].values[idx[0:20]] = True

In [34]:
mask

,0,1,2,3,4
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
145,False,False,False,False,True
146,False,False,False,False,True
147,False,False,False,False,True
148,False,False,False,True,True


In [35]:
mask_final = mask.iloc[:].sum(axis=1).astype(bool)

In [36]:
DF_bias = DF_bias[mask_final]
len(DF_bias), len(mask_final), np.sum(mask_final)

(76, 150, 76)